# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846823238643                   -0.70    4.8
  2   -7.852318991180       -2.26       -1.53    1.0
  3   -7.852613293462       -3.53       -2.56    1.5
  4   -7.852645914062       -4.49       -2.86    2.8
  5   -7.852646447061       -6.27       -3.11    1.0
  6   -7.852646678319       -6.64       -4.04    1.0
  7   -7.852646686684       -8.08       -5.12    2.0
  8   -7.852646686722      -10.41       -5.35    1.8
  9   -7.852646686726      -11.40       -5.49    1.0
 10   -7.852646686730      -11.48       -6.63    1.0
 11   -7.852646686730      -12.98       -6.99    2.2
 12   -7.852646686730      -14.21       -7.98    1.2
 13   -7.852646686730      -14.75       -8.87    2.0
 14   -7.852646686730   +  -15.05       -9.28    2.2
 15   -7.852646686730   +  -15.05       -9.77    1.0
 16   -7.852646686730      -15.05      -10.33    1.5
 17   -7.852646686730   +  -15.05      -10.46 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846808819591                   -0.70           4.8
  2   -7.852528908803       -2.24       -1.63   0.80    2.0
  3   -7.852637552428       -3.96       -2.71   0.80    1.0
  4   -7.852646424450       -5.05       -3.30   0.80    2.0
  5   -7.852646670509       -6.61       -4.13   0.80    1.2
  6   -7.852646686500       -7.80       -4.69   0.80    2.5
  7   -7.852646686726       -9.65       -5.46   0.80    1.2
  8   -7.852646686730      -11.47       -6.67   0.80    1.2
  9   -7.852646686730      -12.53       -7.25   0.80    2.8
 10   -7.852646686730      -14.35       -8.37   0.80    1.0
 11   -7.852646686730   +  -14.45       -8.97   0.80    2.5
 12   -7.852646686730   +    -Inf       -9.85   0.80    1.2
 13   -7.852646686730      -14.45      -10.76   0.80    2.0
 14   -7.852646686730   +    -Inf      -11.17   0.80    2.2
 15   -7.852646686730   +  -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.416073e+01     3.535297e+00
 * time: 0.4308600425720215
     1     1.334425e+00     1.837826e+00
 * time: 0.6270830631256104
     2    -1.942905e+00     2.125159e+00
 * time: 0.6538369655609131
     3    -3.503381e+00     1.869401e+00
 * time: 0.6925251483917236
     4    -4.856005e+00     1.664679e+00
 * time: 0.7308809757232666
     5    -6.460664e+00     1.501403e+00
 * time: 0.7695419788360596
     6    -7.336488e+00     6.378862e-01
 * time: 0.8085031509399414
     7    -7.599174e+00     3.643903e-01
 * time: 0.8354041576385498
     8    -7.716497e+00     1.092967e-01
 * time: 0.8620550632476807
     9    -7.766992e+00     2.183636e-01
 * time: 0.8887979984283447
    10    -7.816382e+00     9.304798e-02
 * time: 0.9153871536254883
    11    -7.839249e+00     7.649475e-02
 * time: 0.9420890808105469
    12    -7.846770e+00     4.907009e-02
 * time: 0.9692130088806152
    13    -7.850171e+00     3.751030e-02
 * time: 0.9959819316

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826838344                   -0.70    4.5
  2   -7.852317049462       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686711                   -2.54
  2   -7.852646686730      -10.72       -5.98
  3   -7.852646686730   +    -Inf      -12.65


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.1214190648982514e-12
|ρ_newton - ρ_scfv| = 1.2941715060539776e-12
|ρ_newton - ρ_dm|   = 4.681922763150632e-10
